In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load data
data = pd.read_csv('melb_data.csv')

# Select relevant columns
selected_columns = ['Rooms', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Price']
data = data[selected_columns]

# Check initial null values
print("Initial Null Values:")
print(data.isnull().sum())

# Handle null values
data = data.dropna(subset=['Price', 'Distance'])
for col in ['Bedroom2', 'Bathroom', 'Car']:
    data[col] = data[col].fillna(data[col].median())

# Verify no null values
print("\nNull Values After Imputation:")
print(data.isnull().sum())

# Remove duplicates
data = data.drop_duplicates()
print(f"\nNumber of rows after removing duplicates: {len(data)}")

# Handle inconsistencies
data = data[(data['Price'] > 0) & (data['Distance'] >= 0) & 
            (data['Bathroom'] >= 0) & (data['Car'] >= 0)]

# Fix Bedroom2 > Rooms
data.loc[data['Bedroom2'] > data['Rooms'], 'Bedroom2'] = data['Rooms']

# Feature engineering: Bedroom_Discrepancy (after correction)
data['Bedroom_Discrepancy'] = data['Rooms'] - data['Bedroom2']

# Verify Bedroom_Discrepancy is non-positive
print("\nBedroom_Discrepancy Statistics Before Outlier Removal:")
print(data['Bedroom_Discrepancy'].describe())

# Remove outliers using IQR for Price
Q1 = data['Price'].quantile(0.25)
Q3 = data['Price'].quantile(0.75)
IQR = Q3 - Q1
data = data[~((data['Price'] < (Q1 - 1.5 * IQR)) | (data['Price'] > (Q3 + 1.5 * IQR)))]

# Cap Rooms and Bedroom2 at 8 (reasonable for most houses)
data = data[(data['Rooms'] <= 8) & (data['Bedroom2'] <= 8)]
data = data[(data['Bathroom'] <= 5) & (data['Car'] <= 5)]

# Normalize numerical features
scaler = StandardScaler()
numerical_features = ['Rooms', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Bedroom_Discrepancy']
data[numerical_features] = scaler.fit_transform(data[numerical_features])

# Summary statistics
print("\nSummary Statistics After Cleaning:")
print(data.describe())

# Save preprocessed data
data.to_csv('preprocessed_clean_v2.csv', index=False)
print("\nPreprocessed data saved to 'preprocessed_clean_v2.csv'")

Initial Null Values:
Rooms        0
Distance     0
Bedroom2     0
Bathroom     0
Car         62
Price        0
dtype: int64

Null Values After Imputation:
Rooms       0
Distance    0
Bedroom2    0
Bathroom    0
Car         0
Price       0
dtype: int64

Number of rows after removing duplicates: 13179

Bedroom_Discrepancy Statistics Before Outlier Removal:
count    13179.000000
mean         0.042795
std          0.241703
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          4.000000
Name: Bedroom_Discrepancy, dtype: float64

Summary Statistics After Cleaning:
              Rooms      Distance      Bedroom2      Bathroom           Car  \
count  1.251800e+04  1.251800e+04  1.251800e+04  1.251800e+04  1.251800e+04   
mean  -1.907193e-16 -2.542923e-16  9.081869e-17  8.173682e-17  1.089824e-16   
std    1.000040e+00  1.000040e+00  1.000040e+00  1.000040e+00  1.000040e+00   
min   -2.045452e+00 -1.718545e+00 -3.105764e+00 -2.353672e+00 -1.774964e+